<a href="https://colab.research.google.com/github/deltorobarba/machinelearning/blob/master/kfp_component_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Set your output and project. !!!Must Do before you can proceed!!!
EXPERIMENT_NAME = 'Tutorial Comp from Scratch V2.51'
PROJECT_NAME =  'erwinh-mldemo'                      #'Your-Gcp-Project-Name'

In [0]:
import kfp
from kfp import gcp
import os
client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)
# Load the component by calling load_component_from_file or load_component_from_url
# To load the component, the pipeline author only needs to have access to the component.yaml file.
# The Kubernetes cluster executing the pipeline needs access to the container image specified in the component.

In [0]:
component_url_prefix = 'https://raw.githubusercontent.com/Huize501/kfp/master/comp-from-scratch/kfp-test-component1/' 


In [0]:
dummy_op = kfp.components.load_component_from_url(component_url_prefix + 'component.yaml')# dummy_op = kfp.components.load_component_from_url('http://....../component.yaml')

In [0]:
component2_url_prefix = 'https://raw.githubusercontent.com/Huize501/kfp/master/comp-from-scratch/kfp-test-component2/'


In [0]:
dummy_op2 = kfp.components.load_component_from_url(component2_url_prefix + 'component2.yaml')

In [0]:
from kfp import dsl

# dummy_op is now a "factory function" that accepts the arguments for the component's inputs
# and produces a task object (e.g. ContainerOp instance).
# Inspect the dummy_op function in Jupyter Notebook by typing "dummy_op(" and pressing Shift+Tab
# You can also get help by writing help(dummy_op) or dummy_op? or dummy_op??
# The signature of the dummy_op function corresponds to the inputs section of the component.
# Some tweaks are performed to make the signature valid and pythonic:
# 1) All inputs with default values will come after the inputs without default values
# 2) The input names are converted to pythonic names (spaces and symbols replaced
#    with underscores and letters lowercased).

# Define a pipeline and create a task from a component:
@kfp.dsl.pipeline(name='KFP test erwinh', description='name')
def my_pipeline(name = dsl.PipelineParam(name='name', value='Erwin')):
    dummy1_task = dummy_op(
        # Input name "Input 1 URI" is converted to pythonic parameter name "input_1_uri"
        name = name,
        gcsbucket ='', # set the gcs bucket
        ).apply(kfp.gcp.use_gcp_secret('user-gcp-sa'))
    
    dummy2_task = dummy_op2(
        gcspath=dummy1_task.outputs['output_path'],
        ).apply(kfp.gcp.use_gcp_secret('user-gcp-sa'))

In [0]:
pipeline_func = my_pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)

In [0]:
#Specify pipeline argument values
arguments = {}

#Submit a pipeline run
run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)

#This link leads to the run information page. 
#Note: There is a bug in JupyterLab that modifies the URL and makes the link stop working

# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.